# Linear Prediction
Linear prediction is a powerful technique in speech processing. We model sampled speech as the output of a linear, slowly time-varying filter which is excited either by quasi-periodic impulses (for voiced speech) or random noise (for unvoiced speech).

The sampled speech is modeled by the following equation

$
\begin{equation}
s[n] = \sum_{k=1}^pa_ks[n-k] + Ge[n]
\end{equation}
$

The vocal tract is then modeled by a system whose $z$-transform is given as

$
H(z)=\frac{S(z)}{E(z)}=\frac{G}{1-\sum_{k=1}^pa_kz^{-k}}
$


### Estimating model parameters
Let us define a signal that follows the linear prediction equation exactly

$
\begin{equation}
\tilde{s}[n] = \sum_{k=1}^p\alpha_ks[n-k]
\end{equation}
$

The prediction error is the difference between $\tilde{s}[n]$ and $s[n]$ given by

$
d[n] = s[n] - \tilde{s}[n] = s[n] - \sum_{k=1}^p\alpha_ks[n-k]
$

### Block processing
To proceed, we will process the speech signal block by block. The blocks are indexed by $\hat{n}$ and will be obtained by windowing the speech signal

$
\begin{equation}
s_{\hat{n}}[m] = w[m]s[m - \hat{n}B]
\end{equation}
$

The parameters will be computed by minimizing the mean square error.

The prediction error for the block samples is

$
\begin{equation}
d_{\hat{n}}[m]= s_{\hat{n}}[m] - \sum_{k=1}^p\alpha_ks_{\hat{n}}[m-k]
\end{equation}
$

We compute the average of the squares of these error values

$\begin{equation}
E_{\hat{n}} = \Big\langle\Big ( s_{\hat{n}}[m] - \sum_{k=1}^p\alpha_ks_{\hat{n}}[m-k]\Big)^2\Big\rangle
\end{equation}
$
where $\langle\rangle$ is the averaging operator.

### Minimum mean square error estimator
To determine the linear prediction coefficients, we seek to minimize the mean square error $E_{\hat{n}}$. We take the derivative with respect to the parameters $\alpha_i$ and set it to zero. 

$\begin{equation}
\frac{\partial E_{\hat{n}}}{\partial\alpha_i }=0\quad i=1,\ldots,p
\end{equation}
$

This yields

$\begin{equation}
\sum_{k=1}^p\alpha_k\langle s_{\hat{n}}[m-i]s_{\hat{n}}[m-k]\rangle=\langle s_{\hat{n}}[m-i]s_{\hat{n}}[m]\rangle\quad i=1,\ldots,p
\end{equation}
$


Or

$\begin{equation}
\sum_{k=1}^p\alpha_k\varphi_{\hat{n}}[i,k]= \varphi_{\hat{n}}[i,0]
\end{equation}
$
where $\varphi_{\hat{n}}[i,k]=\langle s_{\hat{n}}[m-i]s_{\hat{n}}[m-k]\rangle$

We can write this in matrix form as

$
\begin{equation} 
    \begin{bmatrix} 
        \varphi_{\hat{n}}[1,1] & \ldots & \varphi_{\hat{n}}[1,p] \\ 
        \vdots & \vdots & \vdots \\ 
        \varphi_{\hat{n}}[p,1] & \ldots & \varphi_{\hat{n}}[p,p] 
    \end{bmatrix} 
    \begin{bmatrix} \alpha_1 \\ \vdots \\ \alpha_p \end{bmatrix} 
    =
    \begin{bmatrix} \varphi_{\hat{n}}[1,0] \\ \vdots \\ \varphi_{\hat{n}}[p,0] \end{bmatrix} 
\end{equation} 
$

or compactly as 


\begin{equation}
\mathbf{\Phi}\alpha = \psi
\end{equation}


### Computing $\varphi_{\hat{n}}[i,k]$

To proceed we must compute $\varphi_{\hat{n}}[i,k]$ which involves averaging over $s_{\hat{n}}[m-i]s_{\hat{n}}[m-k]$ for $1\leq i\leq p$, $0\leq k \leq p$.

We have

$\begin{equation}
\varphi_{\hat{n}}[i,k] = \sum_{m=-M_1}^{M_2}s_{\hat{n}}[m-i]s_{\hat{n}}[m-k]
\end{equation}
$
for $1\leq i\leq p$, $0\leq k \leq p$.


## Example
Let us explore the computation of linear prediction coefficients in an audio sample. We will use a recording of the word "sita"


In [ ]:
import librosa
import matplotlib.pyplot as plt
import numpy as np
import scipy

signal, sampling_rate = librosa.load('sita.wav', sr=None)

%matplotlib inline
plt.plot(np.arange(len(signal)) / sampling_rate, signal)
plt.xlabel(r'Time(s)')

In [ ]:
# short segment
block_length = 256
segment_time = 1.3
segment = signal[int(sampling_rate * segment_time): int(sampling_rate * segment_time) + block_length ]

In [ ]:
plt.plot(np.arange(len(segment)) / sampling_rate, segment)
plt.xlabel(r'Time(s)')

In [ ]:
#extract lpc
p = 2
a = librosa.lpc(segment, order=p)

# form the filter  H(z)
b = np.hstack([[0], -1 * a[1:]])

#estimate signal
segment_hat = scipy.signal.lfilter(b, [1], segment)

In [ ]:
plt.plot(np.arange(len(segment)) / sampling_rate, segment)
plt.plot(np.arange(len(segment_hat)) / sampling_rate, segment_hat)
plt.xlabel(r'Time(s)')

In [ ]:
# plot the residual
plt.plot(np.arange(len(segment_hat)) / sampling_rate, segment - segment_hat)
plt.xlabel(r'Time(s)')
plt.xlim([.002, .008])

In [ ]:
# plot the STFT and LPC spectrum on the same graph